In [1]:

import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

df=pd.read_csv("firstTry.csv")
df


,predictor,rating_chooses_appropriate_action,rating_commits_to_action,rating_gathers_information,rating_identifies_issues_opportunities,rating_interprets_information,rating_involves_others,rating_decision_making_final_score
0,"Hi Tracy, This sounds very ionteresting, prio...",2.0,2,2,2.0,2,3.0,2
1,"Hi Tracy, I am happy to see we have a self mo...",4.0,3,4,3.0,4,2.0,6
2,J.J please can you work with her for two mor...,2.0,2,2,2.0,2,2.0,2
3,"Hi Paxton Let me talk to Debby first, before...",3.0,3,3,2.0,2,2.0,3
4,JJ HR along with myself and the Final Assem...,3.0,3,2,3.0,2,2.0,3
...,...,...,...,...,...,...,...,...
1461,"Dear Tracey,I wanted to take the time to than...",2.0,2,2,3.0,3,2.0,2
1462,Hi Tracy - thanks for your note and for shari...,4.0,4,3,4.0,4,4.0,6
1463,"Dear Tracey,Thanks for sharing the data.We do...",3.0,3,2,3.0,2,2.0,2
1464,"Hi Tracy, Thanks for sharing customer satisfa...",4.0,3,3,4.0,3,4.0,6


In [11]:
# Criterion 1 model

#Note: Once you find a satisfactory cross-validated score, you don't need to split the data, just run the model on everything
#and predict the validate.csv on the second chunk
y=df['rating_chooses_appropriate_action']
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)

#count_vectorizer = CountVectorizer(stop_words='english')
#count_train = count_vectorizer.fit_transform(X_train.values)
#count_test = count_vectorizer.transform(X_test.values)

#Converting words to numbers
#Note: try other methods if you have time, like sentiment analysis, bag of words, or quanteda. 
#Note: You might need to change max_features if it gives you an error in the next chunk, when predicting the validate dataset
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=13000)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

#Hyperparameter tuning
#Note: use less parameters or make the ranges shorter if it's taking too long
param_grid = {'max_depth':range(1, 7),
                'learning_rate':np.arange(0.005, 0.3, 0.07),
                 'n_estimators':np.arange(100, 2000, 200),
                 'gamma':np.arange(0, 0.06, 0.01),
                 'subsample':np.arange(0.5, 1, 0.1),
                 'colsample_bytree':np.arange(0.5, 1, 0.1)}

#Fitting model
#Note: Hillclimbing methods could be used instead of gridsearch. This might be quicker. 
#Note: randomizedSearchCV() could be used instead of GridSearchCV() to make the algorithm quicker.
#Note: try other models, like neural network classifier. We prefer a classification algorithm instead of regression. 
grid = GridSearchCV(xgb.XGBRegressor(random_state=0), 
                    param_grid=param_grid, cv=StratifiedShuffleSplit(100, random_state=1), verbose=3, scoring='neg_mean_squared_error')
    
grid.fit(tfidf_train, y_train)
pred1 = grid.predict(tfidf_test)

print("Best parameters:", grid.best_params_)
#Note: If you're using a classifier, you're going to have to change the metric to be the AUC or accuracy instead of the RMSE or r2
print("Lowest RMSE: ", (-grid.best_score_)**(1/2.0))
r2_score(y_test, pred1)
#Use this for classification
#metrics.accuracy_score(y_test, pred)

NameError: name 'df' is not defined

In [14]:
#This chunk is predicting the test csv
validation=pd.read_csv("validate.csv")
count= tfidf_vectorizer.fit_transform(validation['predictor'])
pred1 = grid.predict(count)

ValueError: Feature shape mismatch, expected: 19456, got 12474

In [32]:
pred1.round()

array([2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 3., 2., 2., 2., 2., 2.,
       3., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [34]:
count

<487x12000 sparse matrix of type '<class 'numpy.float64'>'
	with 114622 stored elements in Compressed Sparse Row format>

In [2]:
# Criterion 2 model

#Note: Once you find a satisfactory cross-validated score, you don't need to split the data, just run the model on everything
#and predict the validate.csv on the second chunk
y=df['rating_commits_to_action']
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)

#count_vectorizer = CountVectorizer(stop_words='english')
#count_train = count_vectorizer.fit_transform(X_train.values)
#count_test = count_vectorizer.transform(X_test.values)

#Converting words to numbers
#Note: try other methods if you have time, like sentiment analysis, bag of words, or quanteda. 
#Note: You might need to change max_features if it gives you an error in the next chunk, when predicting the validate dataset
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=12474)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

#Hyperparameter tuning
#Note: use less parameters or make the ranges shorter if it's taking too long
param_grid = {
    'hidden_layer_sizes': [(50,100,50), (100,)],
    'activation': ['tanh'],
    'solver': ['sgd'],
    'alpha': [0.0001],
    'learning_rate': ['constant'],
    'max_iter': [500]
}


#Fitting model
#Note: Hillclimbing methods could be used instead of gridsearch. This might be quicker. 
#Note: randomizedSearchCV() could be used instead of GridSearchCV() to make the algorithm quicker.
#Note: try other models, like neural network classifier. We prefer a classification algorithm instead of regression. 
grid = GridSearchCV(MLPClassifier(random_state=0), 
                    param_grid=param_grid, cv=5, verbose=3, scoring='accuracy')
    
grid.fit(tfidf_train, y_train)
pred2 = grid.predict(tfidf_test)

print("Best parameters:", grid.best_params_)
#Note: If you're using a classifier, you're going to have to change the metric to be the AUC or accuracy instead of the RMSE or r2
#print("Lowest RMSE: ", (-grid.best_score_)**(1/2.0))
#r2_score(y_test, pred2)
accuracy = accuracy_score(y_test, pred2)
print("Accuracy score: ", accuracy)
#Use this for classification
#metrics.accuracy_score(y_test, pred)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


c:\Users\miked\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\miked\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/5] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 100, 50), learning_rate=constant, max_iter=500, solver=sgd;, score=0.640 total time=  43.1s


In [5]:
pred2.round()


array([2, 4, 3, 3, 4, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 4, 4,
       3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3,
       3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3,
       3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 3, 3, 4, 4, 4, 3, 3, 3, 4, 3,
       3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3,
       3, 2, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3,
       2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3,
       3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2,
       3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3,
       4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3,

In [10]:
#This chunk is predicting the test csv
validation=pd.read_csv("validate.csv")
count= tfidf_vectorizer.transform(validation['predictor'])
pred_2 = grid.predict(count)

ValueError: X has 12474 features, but MLPClassifier is expecting 13000 features as input.

In [22]:
# Criterion 3 model

y=round(df['rating_gathers_information'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.47368421052631576

In [23]:
# Criterion 4 model

y=round(df['rating_identifies_issues_opportunities'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.6469298245614035

In [24]:
# Criterion 5 model

y=round(df['rating_interprets_information'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.6162280701754386

In [25]:
# Criterion 6 model

y=round(df['rating_involves_others'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.5153508771929824

In [ ]:
# Criterion 7 model

y=round(df['rating_decision_making_final_score'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)